# run hapBLOCK on diploid gt vcf

In [1]:
import sys
sys.path.append('/mnt/archgen/users/yilei/tools/hapBLOCK/python3')
from IO.h5_modify import merge_in_af, get_af, get_af1000G, lift_af, save_h5, get_af1000G_atGivenPos

In [3]:
!for ch in {1..22};do bcftools view -r $ch -m2 -M2 -v snps -O v -o ./vcfbychr/chr$ch.temp.vcf merged.glimpse.GP99.MAC1.transversion_only.vcf.gz; done

In [4]:
import numpy as np
import h5py
import pandas as pd

def merge_in_ld_map(path_h5, path_snp1240k, chs=range(1,23), write_mode="a"):
    """Merge in MAP from eigenstrat .snp file into
    hdf5 file. Save modified h5 in place 
    path_h5: Path to hdf5 file to modify.
    path_snp1240k: Path to Eigenstrat .snp file whose map to use
    chs: Which Chromosomes to merge in HDF5 [list].
    write_mode: Which mode to use on hdf5. a: New field. r+: Change Field"""
    with h5py.File(path_h5, "r") as f:
        print("Lifting LD Map from eigenstrat to HDF5...")
        print("Loaded %i variants." % np.shape(f["calldata/GT"])[0])
        print("Loaded %i individuals." % np.shape(f["calldata/GT"])[1])

        ### Load Eigenstrat
        df_snp = pd.read_csv(path_snp1240k, header=None, sep=r"\s+", engine="python")
        df_snp.columns = ["SNP", "chr", "map", "pos", "ref", "alt"]

        rec = np.zeros(len(f["variants/POS"]))  # Create the array for vector

        for ch in chs:
            df_t = df_snp[df_snp["chr"] == ch]
            print(f"Loaded {len(df_t)} Chr.{ch} 1240K SNPs.")

            idx_f = f["variants/CHROM"][:].astype("str")==str(ch)
            if np.sum(idx_f)==0:  # If no markers found jump to next chromosome
                print("Did not find any markers...")
                continue
            rec_ch = np.zeros(len(idx_f), dtype="float")

            ### Intersect SNP positions
            its, i1, i2 = np.intersect1d(f["variants/POS"][idx_f], df_t["pos"], return_indices=True)

            l = np.sum(idx_f)
            print(f"Intersection {len(i2)} out of {l} HDF5 SNPs")

            ### Extract Map positions
            rec_ch[i1] = df_t["map"].values[i2]  # Fill in the values in Recombination map

            ### Interpolate if Needed (map position still 0)
            itp_idx = (rec_ch == 0)
            if np.sum(itp_idx) > 0:   # In case we have to interpolate
                print(f"Interpolating {np.sum(itp_idx)} variants.")
                x = df_t["pos"] 
                y = df_t["map"]   
                x1 = f["variants/POS"][:][idx_f]  # Extract all positions of interest
                rec_ch = np.interp(x1, x, y) 
            
            ### Make sure that sorted
            assert(np.all(np.diff(rec_ch)>=0))  # Assert the Recombination Map is sorted! (no 0 left and no funky stuff)
            rec[idx_f]=rec_ch # Set the Map position for chromosome indices
            print(f"Finished Chromosome {ch}.")
    
    ### Now create the new column in hdf5
    print("Adding map to HDF5...")
    with h5py.File(path_h5, write_mode) as f0:
        group = f0["variants"]
        l = len(f0["variants/POS"])
        if write_mode == "a":  # If appending new data
            group.create_dataset('MAP', (l,), dtype='f')   
        f0["variants/MAP"][:] = rec[:]
    print("We did it. Finished.")

import allel
def vcf_to_1240K_hdf(in_vcf_path = "/n/groups/reich/ali/WholeGenomeImputation/imputed/v43.4/chr3.bcf",
                     path_vcf = "./data/vcf/1240k_v43/ch3.vcf.gz",
                     path_h5 = "./data/hdf5/1240k_v43/ch3.h5",
                     map_path="/n/groups/reich/DAVID/V43/V43.5/v43.5.snp",
                     ch=3):
    """Convert Ali's vcf to 1240K hdf5. 
    If marker_path empty, no SNP filtering done.
    If map_path empty, no genetic map is merged in.
    """ 
    path_vcf = in_vcf_path # Use the unfiltered input in next step
    
    allel.vcf_to_hdf5(input=path_vcf, output=path_h5, 
                  fields = ['variants/*', 'calldata/*', "samples"], compression="gzip") # Do the conversion to hdf5. Takes hours
    print("Finished conversion to hdf5!")
    
    if len(map_path)>0:
        merge_in_ld_map(path_h5=path_h5, 
                    path_snp1240k=map_path,
                    chs=[ch])

In [ ]:
for ch in range(1,23):
    
    in_vcf_path = f"./vcfbychr/chr{ch}.temp.vcf"
    path_h5 = f"./hdf5/ch{ch}.h5"
    map_path = f"/mnt/archgen/users/hringbauer/data/MinMyc.snp"
    
    vcf_to_1240K_hdf(in_vcf_path = in_vcf_path, path_vcf = "", path_h5=path_h5,
                    map_path = map_path, ch=ch)

    # grabbing 1000G AF
    # with h5py.File(f'/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/chr{ch}.hdf5', 'r') as f_ref:
    #     with h5py.File(path_h5, 'r') as f_target:
    #         pos = np.array(f_target['variants/POS'])
    #         af = get_af1000G_atGivenPos(f_ref, pos)
    # merge_in_af(path_h5, af)

    # with h5py.File(f'./hdf5/ch{ch}.h5', 'a') as f:
    #     # modify GP field to add a little bit uncertainty
    #     nloci, nsample, _ = f['calldata/GT'].shape
    #     gts = np.sum(f['calldata/GT'], axis=2)
    #     assert(np.all(gts <=2))
    #     gps = 0.0005*np.ones((nloci, nsample, 3))
    #     for i in range(nsample):
    #         for j in range(nloci):
    #             gps[j, i, gts[j, i]] = 0.999
    #     group = f['calldata']
    #     group.create_dataset('GP', (nloci, nsample, 3), dtype='f')
    #     f['calldata/GP'][:,:,:] = gps

    print(f"Finished running chromosome {ch}. GZ")

In [7]:
import sys
sys.path.append("/mnt/archgen/users/yilei/tools/hapBLOCK/python3/")     
from run import hapBLOCK_all

hapBLOCK_all(folder_in="./hdf5/ch", iids = [], 
                   chs=range(1,23), folder_out="./hapBLOCK", output=False, prefix_out="", logfile=False,
                   l_model="hdf5", IBD2=True, p_col='default', 
                   ibd_in=1, ibd_out=10, ibd_jump=500, min_cm1=5, min_cm2=2,
                   cutoff_post=0.99, max_gap=0.0075, save=3)

Run hapBLOCK on 4 samples...
IBD1+IBD2 region total length: 109.376
IBD2 region total length: 0.000
IBD1+IBD2 region total length: 93.725
IBD2 region total length: 0.000
IBD1+IBD2 region total length: 101.158
IBD2 region total length: 2.135
IBD1+IBD2 region total length: 111.596
IBD2 region total length: 2.044
IBD1+IBD2 region total length: 86.793
IBD2 region total length: 2.171
IBD1+IBD2 region total length: 96.958
IBD2 region total length: 0.000


# visualize 

In [1]:
import sys
sys.path.append("/mnt/archgen/users/yilei/tools/hapBLOCK/python3/")  

In [2]:
from plot.plot_posterior import plot_posterior_7States_plusGeno
with open('merged.glimpse.GP99.MAC1.transversion_only.seg') as f:
    counter = 1
    for line in f:
        id1, id2, ch, _, _, _, start, end, length, *_ = line.strip().split()
        ch, start, end, length = int(ch), float(start), float(end), float(length)
        plot_posterior_7States_plusGeno(f"./hapBLOCK/{id1}_{id2}/{id1}_{id2}/chr{ch}", start=start - 5, end=end + 5, \
            iids=[id1, id2], path2hdf5="./hdf5/ch", ch=ch, prefix=str(counter), outFolder="./visualize", truth=[start, end])
        counter += 1


Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version


findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
plot IBD new version
Filtering to 0.99 GP variants: 1.000x
Fil

<Figure size 432x288 with 0 Axes>

In [12]:
id1="I3950"
id2="I5273"
ch=10
counter=71
plot_posterior_7States_plusGeno(f"./hapBLOCK/{id1}_{id2}/{id1}_{id2}/chr{ch}", start=170, end=190, \
            iids=[id1, id2], path2hdf5="./hdf5/ch", ch=ch, prefix=str(counter), outFolder="./visualize")

Filtering to 0.99 GP variants: 1.000x
Filtering to 0.99 GP variants: 1.000x
Filtering to common GP variants: 1.000x
[False False False False False False False False False  True  True False
 False False False False  True False False False False False False False
 False False False False  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


<Figure size 432x288 with 0 Axes>